In [1]:
"""
If you want to use new map results, then go to tournament_reduce.py, and replace the extract() method with the following code. 
This will save the map results for use in this notebook. Once those are saved, switch the original code back. 
"""
"""

    async def extract(
        self, pages: list[LookupOutput], district: District, term: str, town: str
    ):
        # We first map extraction across all pages.
        results = []
        empty_results = []
        map_pickle = []
        async for r in super().extract(pages, district, term, town):
            map_pickle.append(r)
            if (r.output is not None) and r.output.extracted_text:
                results.append(r)
            else:
                empty_results.append(r)
                
        # Load existing data if the file already exists
        try:
            with open("./ryurtyn/map_results_11_14_1.dat", "rb") as f:
                existing_data = pickle.load(f)
        except FileNotFoundError:
            existing_data = {}  # If the file doesn't exist, initialize an empty dictionary

        try:
            with open("./ryurtyn/districts_11_14_1.dat", "rb") as f:
                existing_data_district = pickle.load(f)
        except FileNotFoundError:
            existing_data_district = {} 
        # Assuming district.short_name and results are defined somewhere in your code
        new_data = {town+"-"+district.short_name: map_pickle}
        new_data_district = {town+"-"+district.short_name: district}

        # Merge the existing data with the new data
        existing_data.update(new_data)
        existing_data_district.update(new_data_district)

        # Write the updated data back to the file
        with open("./ryurtyn/map_results_11_14_1.dat", "wb") as f:
            pickle.dump(existing_data, f)

        with open("./ryurtyn/districts_11_14_1.dat", "wb") as f:
            pickle.dump(existing_data_district, f)

        # Ensure that we yield one empty result to handle case when the expected output is None
        if len(empty_results) != 0:
            yield empty_results[0]

"""

'\n\n    async def extract(\n        self, pages: list[LookupOutput], district: District, term: str, town: str\n    ):\n        # We first map extraction across all pages.\n        results = []\n        empty_results = []\n        map_pickle = []\n        async for r in super().extract(pages, district, term, town):\n            map_pickle.append(r)\n            if (r.output is not None) and r.output.extracted_text:\n                results.append(r)\n            else:\n                empty_results.append(r)\n                \n        # Load existing data if the file already exists\n        try:\n            with open("./ryurtyn/map_results_11_14_1.dat", "rb") as f:\n                existing_data = pickle.load(f)\n        except FileNotFoundError:\n            existing_data = {}  # If the file doesn\'t exist, initialize an empty dictionary\n\n        try:\n            with open("./ryurtyn/districts_11_14_1.dat", "rb") as f:\n                existing_data_district = pickle.load(f)\n    

In [2]:
import pickle
from zoning.term_extraction.extract.answer_confirm_tester import AnswerConfirmTester, answer_confirm_test
from zoning.term_extraction.eval_results import clean_string_units
from zoning.term_extraction.extract.utils import include_context_around_phrase
import asyncio
import pandas as pd

sk


/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load the map outputs
with open("map_results_11_14_1.dat", "rb") as f:
    map_outputs = pickle.load(f)

# load the dictionary of district objects
with open("districts_11_14_1.dat", "rb") as f:
    districts = pickle.load(f)

In [4]:
gt = pd.read_csv("../data/ground_truth.csv")
gt_first_30 = gt.head(30)

In [5]:
gt_value_exists = gt_first_30[gt_first_30["min_lot_size_gt"].notnull()]
gt_value_not_exist = gt_first_30[gt_first_30["min_lot_size_gt"].isnull()]

In [6]:
# Define an asynchronous function that executes the coroutine object
async def execute_answer_confirm_test(inputs, term, district, town):
    # Your coroutine object (replace this with your actual coroutine)
    async def my_coroutine():
        return await answer_confirm_test(inputs, term, district, 1, town)
    # Execute the coroutine object using await
    return await my_coroutine()

In [7]:
gt_first_30["district"]

0                             Andover Lake
1                             AA Residence
2                   Technology Development
3                          Industrial Park
4                      Restricted Business
5                         General Business
6     Planned Adaptive Reuse Development 3
7                       General Industrial
8                          Village Overlay
9          Housing Opportunity Development
10                              Commercial
11                    I-1 General Industry
12                         General Buiness
13                           Industrial-30
14                           Residence R-3
15                   Multifamily Residence
16                                     A-1
17                              Business C
18          Mixed Use Neighborhood Corners
19                      Mixed Use Corridor
20                              Commercial
21                        Residential R-40
22           Town Center Perimeter Overlay
23         

In [8]:
gt_first_30[gt_first_30["district"] == "Commercial"]["town"].values

array(['bethel', 'bridgewater'], dtype=object)

In [9]:
for d, t in zip(gt_first_30["district"][:5], gt_first_30["town"][:5]):
    print(d, t)

Andover Lake andover
AA Residence ansonia
Technology Development ashford
Industrial Park avon
Restricted Business barkhamsted


In [10]:
total_length = 0
total_correct = 0
total_incorrect = 0

weird_districts = ["I-1 General Industry", "General Buiness"]
null_answer_types = set(["N/A", "None", "Null", "n/a", "none", "null", ""])
# for d in gt_first_30["district"][:5]:
# for d in ["AA Residence", "Technology Development", "Industrial Park", "Restricted Business", "General Business", "Planned Adaptive Reuse Development 3", "General Industrial", "Village Overlay", "Housing Opportunity Development", "Industrial-30"]:
# for d, t in zip(gt_first_30["district"][:15], gt_first_30["town"][:30]):
for t, d in [("avon", "Industrial Park")]:
    town = gt_first_30[gt_first_30["district"] == d]["town"].values[0]
    district_short_name = gt_first_30[gt_first_30["district"] == d]["district_abb"].values[0]
    district = districts[town+"-"+district_short_name]
    inputs = map_outputs[town+"-"+district_short_name]
    town = gt_first_30[gt_first_30["district"] == d]["town"].values[0]
    term = "min_lot_size"
    print("District: ", district)

    true_val = gt_first_30[gt_first_30["district"] == d]["min_lot_size_gt"].values[0]
    try:
        true_val = int(true_val)
    except ValueError:
        print("skipping ", d, "true value is", true_val)
        true_val = ""
        # continue
    true_indices = []
    for a in range(len(inputs)):
        i = inputs[a]
        if i.output and i.output.answer:
            # print(i.output.answer)
            i_answer_clean = clean_string_units(i.output.answer)
            print("GT answer: ", true_val, "Cleaned Answer: ", i_answer_clean, "Raw Answer: ", i.output.answer)
            if i_answer_clean:
                i_answer_clean = int(i_answer_clean[0])
                if i_answer_clean == true_val:
                    true_indices.append(a)
                    continue
            if (not true_val) and (i.output.answer in null_answer_types):
                true_indices.append(a)
                continue
    
    num_correct = 0
    num_incorrect = 0
    curr_length = 0
    
    for a in range(len(inputs)):
        i = inputs[a]
        if i.output and i.output.extracted_text:
            curr_length += 1
            
            res = await execute_answer_confirm_test(i, term, district, town)
            if a in true_indices:
                if res == "Y":
                    num_correct += 1
                else:
                    num_incorrect += 1
            else:
                if res == "N":
                    num_correct += 1
                else:
                    num_incorrect += 1
            print("ANSWER: ", a in true_indices, res)
    total_correct += num_correct
    total_incorrect += num_incorrect
    total_length += curr_length
                    

    print("Number Correct: ", num_correct)
    print("Number Incorrect: ", num_incorrect)
    print("Accuracy: ", num_correct / curr_length)

print("Final Num Correct: ", total_correct)
print("Final Num Incorrect: ", total_incorrect)
print("Final Accuracy: ", total_correct / total_length)

District:  full_name='Industrial Park' short_name='IP'
skipping  Industrial Park true value is nan
GT answer:   Cleaned Answer:  [217800.0] Raw Answer:  5 acres
GT answer:   Cleaned Answer:  [40000.0] Raw Answer:  40,000 sq ft
GT answer:   Cleaned Answer:  [217800.0] Raw Answer:  5 acres
GT answer:   Cleaned Answer:  [30000.0] Raw Answer:  30,000 sq ft
GT answer:   Cleaned Answer:  [40000.0] Raw Answer:  40,000 sq ft
GT answer:   Cleaned Answer:  [30000.0] Raw Answer:  30,000 sq ft
GT answer:   Cleaned Answer:  [1306800.0] Raw Answer:  30 acres


/Users/rusy/Documents/Cornell/fa23/Zoning/zoning-gpt/zoning/term_extraction/extract/utils.py:197: UserWarning: Phrase CELL (1, 1): 
Minimum lot area was found more than once in the document.
  warnings.warn(f"Phrase {phrase} was found more than once in the document.")


# Instructions

You are an expert architectural lawyer. You are looking for facts inside a
document about a Zoning District with the name "Industrial Park" and with an
abbreviated name "IP".

You have identified a potential value for "min_lot_size", which also goes by the following other names: min lot size, min lot coverage, min parcel size, min lot and building, area and bulk, min dimensional, min lot area, lot area, min area, min lot area, min lot and area, min lot, min lot requirements, min parcel area, lot size, lot and building, dimensional, min lot, lot requirements, lot and building requirements, area requirements, area and bulk requirements, dimensional requirements. You will be given a possible value for min_lot_size. You will also be given a rationale for the answer as well as an extracted chunk of text relevant to the answer. Finally, you will be shown the page contents from the document. Your job is to decide whether or not the value is the correct answer for "min_lot_size

tldr: if the answer is "N" we should return a null answer entry 